# Model Benchmark

This Jupyter Notebook aims to create a benchmark model using a Convolutional Neural Network (CNN) with the ResNet50 architecture. Unlike the main model, this benchmark model will not utilize transfer learning. By implementing the same architecture as the main model, we can establish a performance baseline for comparison and evaluate the impact of transfer learning on model performance. Throughout this notebook, we will train the benchmark model from scratch using the available dataset, analyze its accuracy and loss, and assess its performance as a reference point for further improvements.

## 1. Imports

In [1]:
import os
import json
import boto3
import random
import shutil

from tqdm import tqdm

import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.analytics import HyperparameterTuningJobAnalytics
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner, IntegerParameter
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.debugger import Rule, rule_configs, ProfilerRule, DebuggerHookConfig, CollectionConfig, ProfilerConfig, FrameworkProfile
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

from IPython.display import Image, display

[2023-06-30 16:09:01.012 Lucass-MacBook-Air.local:2173 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


## 2. Training

In [2]:
role = "arn:aws:iam::860372235077:role/service-role/AmazonSageMaker-ExecutionRole-20230609T125000"
sagemaker_session = sagemaker.Session()

In [3]:
s3_bucket = 'object-counter-project'
s3_prefix = 'data'
input_path = f's3://{s3_bucket}/{s3_prefix}'
print(f'Input path: {input_path}')

Input path: s3://object-counter-project/data


In [11]:
hyperparameters = {"batch_size": 32, 
                   "learning_rate": 0.001, 
                   "epochs":10}

estimator = PyTorch(entry_point='benchmark_train.py',
                    base_job_name='object-counter-benchmarking',
                    role=role,
                    framework_version='2.0.0',
                    py_version='py310',
                    instance_count=1,
                    instance_type='ml.g4dn.2xlarge',
                    hyperparameters=hyperparameters,
                    output_path=f's3://{s3_bucket}/training_output')

In [12]:
estimator.fit({'training': input_path}, wait=True, logs='All')

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: object-counter-benchmarking-2023-06-30-19-52-23-819


2023-06-30 19:52:26 Starting - Starting the training job...
2023-06-30 19:52:42 Starting - Preparing the instances for training......
2023-06-30 19:53:53 Downloading - Downloading input data............
2023-06-30 19:55:54 Training - Downloading the training image.........
2023-06-30 19:57:24 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-30 19:57:55,425 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-30 19:57:55,444 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-06-30 19:57:55,453 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-06-30 19:57:55,455 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-06-30 19:57:57,041 sagemaker-training-toolkit INFO     No Neurons detected (no

## 3. Results

| Class   | Precision | Recall   | F1      |
|---------|-----------|----------|---------|
| Class 1 | 0.0000    | 0.0000   | 0.0000  |
| Class 2 | 0.0000    | 0.0000   | 0.0000  |
| Class 3 | 0.2563    | 1.0000   | 0.4080  |
| Class 4 | 0.0000    | 0.0000   | 0.0000  |
| Class 5 | 0.0000    | 0.0000   | 0.0000  |

| Average Test Accuracy | 0.2563         |
|----------------------|----------------|
| Average Test Loss     | 1.5794         |